In [1]:
from copydetect import CopyDetector
import csv
import json
import os
import shutil

## --- PARAMETER ---
CONTEST = "BW134" # Contest name
CONTEST_URL = "https://leetcode.com/contest/biweekly-contest-134/" # Contest URL
QUESTION = '/Q3'
LANG = "cpp" # the file extension gonna deal with
THRESHOLD = 0.9 # Merge only if Both A is THRESHOLD% similar to B, and B is THRESHOLD% similar to B

In [ ]:
OUTPUT_HTML = CONTEST+QUESTION+"_"+LANG+".html" # the output HTML generated by the detection tool

detector = CopyDetector(test_dirs=[CONTEST+QUESTION], extensions=[LANG], display_t=0.95, out_file=OUTPUT_HTML)
detector.run()
copied_code = detector.get_copied_code_list()
detector.generate_html_report()
print(len(copied_code))

In [ ]:
def Manual_add(user1, user2, cheater):
    group_1 = -1;
    group_2 = -1;
    
    for j, group in enumerate(cheater):
        if user1 in group:
            group_1 = j
        if user2 in group:
            group_2 = j
    
    if group_1 == -1 and group_2 == -1:
        cheater.append([user1, user2])
    elif group_1 == -1:
        cheater[group_2].append(user1)
    elif group_2 == -1:
        cheater[group_1].append(user2)
    elif group_1 != group_2:
        cheater[group_1].extend(cheater[group_2])
        del cheater[group_2]

In [ ]:
# The cheater group found so far
JSON = CONTEST+QUESTION+"_"+LANG+".json"

with open(JSON) as f:
    cheater = json.load(f)
print(cheater)

In [ ]:
## --- PARAMETER ---
IGNORE = [1, 2]
# Manual_add(1, 2, cheater) # Manually merge cheater

for row in copied_code:
    p1 = row[0]
    p2 = row[1]
    user1 = int(row[2][row[2].rindex("\\")+1:-len(LANG)-1])
    user2 = int(row[3][row[3].rindex("\\")+1:-len(LANG)-1])

    if user1 in IGNORE or user2 in IGNORE:
        continue

    group_1 = -1;
    group_2 = -2;
    
    for j, group in enumerate(cheater):
        if user1 in group:
            group_1 = j
        if user2 in group:
            group_2 = j        

    if p1 >= THRESHOLD and p2 >= THRESHOLD:
        if group_1 == -1 and group_2 == -2:
            cheater.append([user1, user2])
        elif group_1 == -1:
            cheater[group_2].append(user1)
        elif group_2 == -2:
            cheater[group_1].append(user2)
        elif group_1 != group_2:
            cheater[group_1].extend(cheater[group_2])
            del cheater[group_2]
    elif group_1 != group_2:
        print(f"Test file: {row[2]} ({row[0]*100:.2f}%)\n"
              f"Reference file: {row[3]} ({row[1]*100:.2f}%)\n"
              f"Token overlap: {row[6]}")

print("[")
for i in cheater:
    i.sort()
    print(i, ",")
print("]")

JSON = CONTEST+QUESTION+"_"+LANG+".json"
with open(JSON, "w") as f:
    json.dump(cheater, f)

In [ ]:
# # reference
# QUESTION = '/Q3/'

# ref = [424, 725, 895, 932, 949, 950, 970, 1668, 2068, 2183, 2238, 2279, 2315, 2467, 2511, 2596, 2660, 2781, 2799, 2867, 2947, 2957, 2960, 3199, 3225, 3233, 3244, 3271, 3300, 3534, 3546, 3562]
# try:
#     shutil.rmtree(CONTEST+"/rf")
# except:
#     pass
# try:
#     os.mkdir(CONTEST+"/rf")
#     os.mkdir(CONTEST+"/rf/0001-1000")
#     os.mkdir(CONTEST+"/rf/1001-2000")
#     os.mkdir(CONTEST+"/rf/2001-3000")
#     os.mkdir(CONTEST+"/rf/3001-4000")
#     os.mkdir(CONTEST+"/rf/4001-5000")
#     os.mkdir(CONTEST+"/rf/5001-6000")
#     os.mkdir(CONTEST+"/rf/6001-7000")
#     os.mkdir(CONTEST+"/rf/7001-8000")
# except:
#     pass

# for user in ref:
#     folder = str(user//1000)+"001-"+str(user//1000+1)+"000/"

#     shutil.copyfile(CONTEST+QUESTION+folder+str(user)+LANG, CONTEST+"/rf/"+folder+str(user)+LANG)

In [ ]:
# # Not necessary. Copy to test for further testing the first file of each group
# try:
#     shutil.rmtree(CONTEST+"/ts")
# except:
#     pass
# # create folder
# try:
#     os.mkdir(CONTEST+"/ts")
# except:
#     pass
# for i in range(12):
#     try:
#         os.mkdir(CONTEST+"/ts/"+str(i)+"000-"+str(i)+"999")
#     except:
#         pass
#     try:
#         os.mkdir(CONTEST+"/ts/"+str(i)+"000-"+str(i)+"999")
#     except:
#         pass

# for group in cheater:
#     folder = '/'+str(group[0]//1000)+"000-"+str(group[0]//1000)+"999/"

#     shutil.copyfile(CONTEST+QUESTION+folder+str(group[0])+'.'+LANG, CONTEST+"/ts"+folder+str(group[0])+'.'+LANG)

In [ ]:
## generate markdown, and record the rank, user name, and submission ID of the mass cheating group to csv

## --- PARAMETER ---
Q = 3 # i.e. 4 means Q4
CSV = CONTEST+"/Q3_py.csv" # the name of csv gonna be created to record the mass cheating group
# the mass cheating group
cheater = [1, 2]

print("how many cheater: ", len(cheater))

with open(CONTEST+"/"+CONTEST+".csv") as f:
    rows = list(csv.reader(f))
    
    with open(CSV, "w") as csv_:
        for i in cheater:
            print(str(i)+"," + rows[i][0]+"," + rows[i][Q], file=csv_)

    with open(CSV[:-4]+".md", "w") as md:
        print("| rank | user | Q"+str(Q)+"   |\n| ---- | ---- | ---- |", file=md)
    
        for i in cheater:
            print("|", i, "| ["+rows[i][0]+"](https://leetcode.com/u/"+rows[i][0]+") |", CONTEST_URL+"submissions/detail/"+rows[i][Q], "|", file=md)